In [15]:
from FUNCIONES_FEATURES import *
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics as sm
from scipy.stats import pearsonr
from itertools import chain, combinations
import itertools

# DEFINICION

- varianza - 1


- mean_absolute - 2


- median_absolute - 3


- average_range - 4


- coefficient_variation - 5


- quartile_deviation - 6


- mean_deviation - 7


- root_square_deviation - 8


- median_deviation_from_the_mean - 9


- geometric_std - 10


- winsorized_variance - 11


- robust_coefficient_variation - 12

In [16]:
short_window=100
long_window=300

In [17]:
dispersiones = [
    varianza,
    mean_absolute,
    median_absolute,
    average_range,
    coefficient_variation,
    quartile_deviation,
    mean_deviation,
    root_square_deviation,
    median_deviation_from_the_mean,
    geometric_std,
    winsorized_variance,
    robust_coefficient_variation,
]

In [18]:
def power_set(iterable):
    s = list(iterable)
    return list(
        itertools.chain.from_iterable(
            itertools.combinations(s, r) for r in range(len(s) + 1)
        )
    )

def generacion_score_spread_general(data_trading,data_spread_rate,lista_metricas):
    dataframes = {}
    for i in lista_metricas:
        score_ts = scoring(rates_spread_ts=data_spread_rate, short_w=short_window, long_w=long_window, dispersion_function=i)
        spreads_score_df = scoring_vs_max_spread(ts_trading=data_trading, scoring_ts=score_ts, filtracion_n_trades=1000)

        dataframes[i] = spreads_score_df
    return dataframes

In [19]:
AL30_TRADING=pd.read_csv("/Users/rogarciag/Desktop/AI_&_TRADING/ALGORITHMIC_TRADING/PRINCIPAL/NASINI/TRABAJOS_GENERAL/PROYECTO_TRADING/BASES_DE_DATOS/DATOS_TRADING_SPREADS/DATOS_SIN_CAUSION/spread_trading_AL30_CAUSION_ESTABLE.csv")
AL30_SPREADS=pd.read_csv('/Users/rogarciag/Desktop/AI_&_TRADING/ALGORITHMIC_TRADING/PRINCIPAL/NASINI/TRABAJOS_GENERAL/PROYECTO_TRADING/BASES_DE_DATOS/DATOS_TRADING_SPREADS/DATOS_SIN_CAUSION/ts_rates_spreads_AL30_CAUSION_ESTABLE.csv')

In [20]:
AL30_TRADING=AL30_TRADING[0:5000]
AL30_SPREADS=AL30_SPREADS[0:5000]

In [21]:
dataframes=generacion_score_spread_general(data_trading=AL30_TRADING,data_spread_rate=AL30_SPREADS,lista_metricas=dispersiones)

In [22]:
varianza_df=dataframes[varianza]["Score_Inicio_Trade"]
mean_absolute_df=dataframes[mean_absolute]["Score_Inicio_Trade"]
median_absolute_df=dataframes[median_absolute]["Score_Inicio_Trade"]
average_range_df=dataframes[average_range]["Score_Inicio_Trade"]
coefficient_variation_df=dataframes[coefficient_variation]["Score_Inicio_Trade"]
mean_deviation_df=dataframes[mean_deviation]["Score_Inicio_Trade"]
root_square_deviation_df=dataframes[root_square_deviation]["Score_Inicio_Trade"]
median_deviation_from_the_mean_df=dataframes[median_deviation_from_the_mean]["Score_Inicio_Trade"]
geometric_std_df=dataframes[geometric_std]["Score_Inicio_Trade"]
winsorized_variance_df=dataframes[winsorized_variance]["Score_Inicio_Trade"]
robust_coefficient_variation_df=dataframes[robust_coefficient_variation]["Score_Inicio_Trade"]

In [23]:
input_series = [
    varianza_df,
    mean_absolute_df,
    median_absolute_df,
    average_range_df,
    coefficient_variation_df,
    mean_deviation_df,
    root_square_deviation_df,
    median_deviation_from_the_mean_df,
    geometric_std_df,
    winsorized_variance_df,
    robust_coefficient_variation_df,
]



indexed_input_series = list(enumerate(input_series, 1))
power_set_series = power_set(indexed_input_series)

sums_df = pd.DataFrame()
for i, combination in enumerate(power_set_series, 1):
    indices = [str(index) for index, _ in combination]
    column_name = "_".join(indices)
    if not column_name:
        column_name = "empty"
    sums_df[f'combination_{column_name}'] = sum(series for _, series in combination)
sums_df["Target"]=dataframes[varianza]["Maximum Abs Value"]

In [24]:
df=sums_df
df.drop(columns="combination_empty",inplace=True)

# PARAMETRO TRADING -0.06=<   <0.03 

In [25]:
# Assuming your data frame is named df
combination_columns = [col for col in df.columns if 'combination' in col]

r2_scores = {}
correlation_coefficients = {}

for combination_col in combination_columns:
    x = df[combination_col]
    x = np.array(x).reshape(-1, 1)
    y = df['Target']

    sc = StandardScaler().fit(x)
    x_sc = sc.transform(x)

    lm = LinearRegression()
    lm.fit(x_sc, y)
    predictions = lm.predict(x_sc)
    correlation, _ = pearsonr(df[combination_col], df['Target'])

    r2_scores[combination_col] = sm.r2_score(y, predictions)
    correlation_coefficients[combination_col] = correlation

show_top_score=10
sorted_r2_scores = sorted(r2_scores.items(), key=lambda x: x[1], reverse=True)[:show_top_score]
sorted_correlation_coefficients = sorted(correlation_coefficients.items(), key=lambda x: abs(x[1]), reverse=True)[:show_top_score]

print("Top 5 features con mayor r2 positivo:")
for col, r2 in sorted_r2_scores:
    print(f"{col}: {r2:.2f}")

print("\nTop 5 features con mayor correlación absoluta:")
for col, corr in sorted_correlation_coefficients:
    print(f"{col}: {corr:.2f}")


Top 5 features con mayor r2 positivo:
combination_4_5: 0.18
combination_4_5_11: 0.18
combination_4_11: 0.15
combination_4_5_9_11: 0.13
combination_4: 0.13
combination_4_5_9: 0.12
combination_3_4_5_11: 0.11
combination_4_5_8_11: 0.11
combination_5_11: 0.09
combination_3_4_5_9_11: 0.07

Top 5 features con mayor correlación absoluta:
combination_4_5: 0.43
combination_4_5_11: 0.42
combination_4_11: 0.39
combination_4_5_9_11: 0.36
combination_4: 0.36
combination_4_5_9: 0.34
combination_3_4_5_11: 0.33
combination_4_5_8_11: 0.33
combination_5_11: 0.30
combination_3_4_5_9_11: 0.27


# CONCLUSION

Con los datos hasta ahora la mejor combinacion en correlacion y r2 es la misma con las caracteristicas que maximizan esta metrica:

- varianza - 1


- mean_absolute - 2


- average range - 4


- quartile deviation - 6


- winsorized_variance - 11